<a href="https://colab.research.google.com/github/camparchimedes/runaway_robots/blob/main/%F0%9D%9F%B0_bit_%F0%9D%97%9F%F0%9D%97%9F%F0%9D%97%A0_%F0%9D%97%A4%E1%B5%A4%E2%82%90%E2%82%99%E2%82%9C%E1%B5%A2%E1%B6%BB%E2%82%90%E2%82%9C%E1%B5%A2%E2%82%92%E2%82%99_%F0%9D%98%84%F0%9D%97%B6%F0%9D%98%81%F0%9D%97%B5_%F0%9D%97%9A%F0%9D%97%A3%F0%9D%97%A7%F0%9D%97%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AutoGirl_*4-bit*.ches**

> Created by [switchdev](https://github/camparchimedes)


In [ ]:
# @title # ⚡ **AutoGirl_*4-bit*.ches**


# @markdown Quantization methods: `4-bit`, `gptq`

# @markdown The `token` corresponds to the name of the secret that stores your [Hugging Face access token](https://huggingface.co/settings/tokens) in Colab.





In [ ]:
# @markdown ### ⚡ Quantization of......
MODEL_ID = "norallm/normistral-7b-warm" # @param {type:"string"}

# @markdown ---


!BUILD_CUDA_EXT=0 pip install -q auto-gptq transformers
# @markdown ### 🤗 Hugging Face

!pip install -q huggingface_hub
username = "camparchimedes" # @param {type:"string"}
token = "HF_TOKEN" # @param {type:"string"}

MODEL_NAME = MODEL_ID.split('/')[-1]

# Download model
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

import random

from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import load_dataset
import torch
from transformers import AutoTokenizer


# Define base model and output directory
model_id = "norallm/normistral-7b-warm"
#out_dir = model_id + "-GPTQ"

# Load quantize config, model and tokenizer
quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    damp_percent=0.01,
    desc_act=False,
)
model = AutoGPTQForCausalLM.from_pretrained(model_id, quantize_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

n_samples = 1024
data = load_dataset("allenai/c4", data_files="en/c4-train.00001-of-01024.json.gz", split=f"train[:{n_samples*5}]")
tokenized_data = tokenizer("\n\n".join(data['text']), return_tensors='pt')

# Format tokenized examples
examples_ids = []
for _ in range(n_samples):
    i = random.randint(0, tokenized_data.input_ids.shape[1] - tokenizer.model_max_length - 1)
    j = i + tokenizer.model_max_length
    input_ids = tokenized_data.input_ids[:, i:j]
    attention_mask = torch.ones_like(input_ids)
    examples_ids.append({'input_ids': input_ids, 'attention_mask': attention_mask})

%%time

# Quantize with GPTQ
model.quantize(
    examples_ids,
    batch_size=1,
    use_triton=True,
)

# Save model and tokenizer

from huggingface_hub import create_repo, HfApi
from google.colab import userdata, runtime

# Defined in the secrets tab in Google Colab

hf_token = userdata.get('HF_TOKEN')
api = HfApi()

# Create empty repo
create_repo(
    repo_id = f"{username}/{MODEL_NAME}-GPTQ",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)

# Upload gptq files
api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GPTQ",
    allow_patterns=["*.gptq","$.md"],
    token=hf_token
)

model.save_quantized(api.upload_folder, use_safetensors=True)
tokenizer.save_pretrained(api.upload_folder)




Git LFS initialized.
fatal: destination path 'normistral-7b-warm' already exists and is not an empty directory.


config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Reload model and tokenizer
model = AutoGPTQForCausalLM.from_quantized(
    out_dir,
    device=device,
    use_triton=True,
    use_safetensors=True,
)
tokenizer = AutoTokenizer.from_pretrained(api.upload_folder) # out_dir

from transformers import pipeline

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
result = generator("Veit da faen...", do_sample=True, max_length=50)[0]['generated_text']
print(result)

In [ ]:


# Kill runtime
runtime.unassign()